In [1]:
from sklearn import preprocessing
import pandas as pd

In [2]:
df = pd.read_csv('./attacks.csv',index_col=0)

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,2,3,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/lib/arraysetops.py:569: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
Y = df.iloc[:, 8]
X = df.iloc[:, 0: 8]

In [7]:
X = df.iloc[:,0: 8].apply(
    lambda x: " ".join(x.astype(str)), axis=1)

In [8]:
len(X)

5378578

In [15]:
data = pd.concat([Y, X], axis = 1)

In [27]:
data[0].value_counts();

In [28]:
import numpy as np
N = 3
print(data.shape)
row_reduce = data
for i in range (0, N):
    remove_n = row_reduce.shape[0] // 2
    drop_indices = np.random.choice(row_reduce.index, remove_n, replace=False)
    row_reduce = row_reduce.drop(drop_indices)

df = row_reduce

(5378578, 2)


In [10]:
# The maximum number of words to be used. (most frequent)
MAX_NB_WORDS = 674162
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 8
# This is fixed.
EMBEDDING_DIM = 100

In [11]:
# # import tensorflow as tf
# from tf.keras.preprocessing.text import Tokenizer
import tensorflow
tokenizer = tensorflow.keras.preprocessing.text.Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(data[0].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 258 unique tokens.


In [18]:
X = tokenizer.texts_to_sequences(data[0].values)
X = tensorflow.keras.preprocessing.sequence.pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print('Shape of data tensor:', X.shape)

Shape of data tensor: (5378578, 8)


In [19]:
Y = pd.get_dummies(data['Flag']).values
print('Shape of label tensor:', Y.shape)

Shape of label tensor: (5378578, 5)


In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.20, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(4302862, 8) (4302862, 5)
(1075716, 8) (1075716, 5)


In [41]:
model = tensorflow.keras.models.Sequential()
model.add(tensorflow.keras.layers.Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X.shape[1]))
model.add(tensorflow.keras.layers.SpatialDropout1D(0.2))
model.add(tensorflow.keras.layers.LSTM(500, dropout=0.2, recurrent_dropout=0.2, return_sequences=True ))
model.add(tensorflow.keras.layers.LSTM(500, dropout=0.2, recurrent_dropout=0.2))
model.add(tensorflow.keras.layers.Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

epochs = 1
batch_size = 50

history = model.fit(X_train, 
                    Y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_split=0.5,
                    callbacks=[tensorflow.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

Train on 2151431 samples, validate on 2151431 samples
   4500/2151431 [..............................] - ETA: 11:52:50 - loss: 0.3935 - acc: 0.8387

KeyboardInterrupt: 